In [8]:
import os , re, os, datetime, dateutil, pytz
from dateutil import parser
import ACILib

logfile = '/home/jovyan/HTTSDashboard/logs/RabbitMQ_ACI_Event.log'
logfiletime = os.path.getmtime(logfile)
logfiledatetime = datetime.datetime.fromtimestamp(logfiletime)
logfiledatetime

datetime.datetime(2020, 5, 21, 5, 32, 9, 138904)

In [9]:
#It was exected in the jupyter notebook container , path is different from flask container.

date = '2020-02-27'
date = datetime.datetime.today().strftime("%Y-%m-%d")
#current_hour = datetime.datetime.now().hour
#if current_hour >= 6:
#    exit()

jupyter_container_path = '/home/jovyan'
flask_container_path = ''
current_container_path = jupyter_container_path

sourcefile = current_container_path + '/HTTSDashboard/logs/RabbitMQ_ACI_Event.log'
sourcefiletime = os.path.getmtime(sourcefile)
sourcefiledatetime = datetime.datetime.fromtimestamp(sourcefiletime)
sourcefileformattime = sourcefiledatetime.strftime("%Y-%m-%d %H:%M:%S")

commentfile = current_container_path + '/HTTSDashboard/logs/ACI/data/comment.txt'
casestatsdir = current_container_path + '/HTTSDashboard/logs/ACI/data/'
engrstatsdir = current_container_path + '/HTTSDashboard/logs/ACI/data/'
dailyreportfiledir = current_container_path + '/HTTSDashboard/logs/ACI/data/report/'
dailyreportfilename = date+'.txt'
dailyreportstring = ""


SYDEngrList = ['tonzeng','minkwong','siddhp','wilchong','debabbar','lindawa','junwa','annelso2','ikarvoun']
BGLEngrList = ['deepaky','knagavol','jawalia','prpratee','maveer','raghb','roagraw2','shparanj','vkalmath']
BGLOtherEngrList = ['anirukas','reperuma','deepakba','bharatkc','kdoodi']
SYDOtherEngrList = ['zdazhi','zmeng']
CSSEngrList = []
OnShiftEngrList = []

ACI_InterestQueueName = ['WW-ACI-Solutions','CX-APJC-BLR-ACI-SSPT','CX-APJC-SYD-ACI-SSPT','WW-Rakuten-ACI']
SV_InterestQueueName = ['WW-SV','WW-STORAGE']
InterestQueueName = ACI_InterestQueueName

CaseQty = {'CX-APJC-BLR-ACI-SSPT':0,
           'WW-ACI-Solutions':0,
           'CX-APJC-SYD-ACI-SSPT':0,
           'WW-Rakuten-ACI':0,
           'FTS':0,'UC':0}
CaseTakenDic = {}
CaseTakenDicInclNo = {'SYD':{},'SYD_Other':{},'BGL':{},'BGL_Other':{}}
CaseTakenByEngrDic = {}
CaseStatsDic = {"SydQueueVol":0,"BGLQueueVol":0,
                "SydTakenVol":0,"BGLTakenVol":0,
                'FTSCaseVol':0,'UCCaseVol':0,
                "SydQueueVolByPri":[0,0,0,0,0,0],"BGLQueueVolByPri":[0,0,0,0,0,0], #0 FTS #1,2,3,4 Priority #5 Urgent Collab
                "SydTakenVolByPri":[0,0,0,0,0,0],"BGLTakenVolByPri":[0,0,0,0,0,0], #0 FTS #1,2,3,4 Priority #5 Urgent Collab
                "SydOnShiftEngr":1,"BGLOnShiftEngr":1,
                "SydOnShiftEngrTakenVol":0,"BGLOnShiftEngrTakenVol":1,
                "SYDHelper":0,"BGLHelper":0,
                }

with open(sourcefile) as f:
    allevents = f.read()
f.close()

allevents = allevents.split("\n")
allevents[-10:]

['RabbitMQ_ACI_Event.log-2020-05-21 04:35:25,904-INFO-MQToRedis:689134352 3 Case_Accepted Fault code F1527 taken by maveer APT-ACI-SOLUTIONS2 at 2020-05-21 04:32:47',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:36:21,373-INFO-MQ:689134352 3 Case_Created CX-APJC-BLR-ACI-SSPT Fault code F1527 by NULL at 2020-05-21 04:31:22',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:38:54,680-INFO-MQ:689134352 3 Case_Created CX-APJC-BLR-ACI-SSPT Fault code F1527 by NULL at 2020-05-21 04:31:22',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:41:57,424-INFO-MQ:689134352 3 Case_Created CX-APJC-BLR-ACI-SSPT Fault code F1527 by NULL at 2020-05-21 04:31:22',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:45:23,893-INFO-MQ:689134352 3 Case_Created CX-APJC-BLR-ACI-SSPT Fault code F1527 by NULL at 2020-05-21 04:31:22',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:51:32,623-INFO-MQ:689134406 3 Case_Created CX-APJC-BLR-ACI-SSPT Tacacs+ Provider unreachable by NULL at 2020-05-21 04:50:01',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:52:54,962-INFO-MQ

In [10]:
def GetRawEventsFromDate(allevents,date='2020-02-27',shift_start_hour=0,shift_end_hour=6,debug=False):
    
    date = date
    shift_start_hour = shift_start_hour
    shift_end_hour = shift_end_hour
    allevents_in_date = []
    
    CaseMatchLine_Reg = re.compile(r'.*?MQToRedis:\d{9}\s\d\s.*at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    
    for line in allevents[:]:
        if date not in line:
            continue
        if re.search(CaseMatchLine_Reg,line):
            line_datetime = parser.parse(re.search(CaseMatchLine_Reg,line).group(1))
            #dispatched case might dispatch to today while not take into account
            
            if line_datetime.strftime("%Y-%m-%d") != date:
                #print("{} - {}".format(line_datetime.strftime("%Y-%m-%d"),date))
                continue
            if line_datetime.hour >= shift_start_hour and line_datetime.hour < shift_end_hour:
                allevents_in_date.append(line)
    return allevents_in_date

allevents_in_date = GetRawEventsFromDate(allevents,date=date)
allevents_in_date[-10:]

['RabbitMQ_ACI_Event.log-2020-05-21 04:23:42,216-INFO-MQToRedis:689134216 3 Case_InQueue CX-APJC-BLR-ACI-SSPT IM121875082/BBSRCPTCCESB01/ACI/3.2(3n)/BFD flapped between BL 1/2 and SAR 1/2 for all links at 2020-05-21 04:21:56',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:24:58,666-INFO-MQToRedis:689134216 3 Case_InQueue CX-APJC-BLR-ACI-SSPT IM121875082/BBSRCPTCCESB01/ACI/3.2(3n)/BFD flapped between BL 1/2 and SAR 1/2 for all links at 2020-05-21 04:21:59',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:28:05,334-INFO-MQToRedis:689134216 3 Case_Accepted IM121875082/BBSRCPTCCESB01/ACI/3.2(3n)/BFD flapped between BL 1/2 and SAR 1/2 for all links taken by raghb APT-ACI-SOLUTIONS2 at 2020-05-21 04:26:35',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:32:23,480-INFO-MQToRedis:689134352 3 Case_InQueue CX-APJC-BLR-ACI-SSPT Fault code F1527 at 2020-05-21 04:31:28',
 'RabbitMQ_ACI_Event.log-2020-05-21 04:35:02,369-INFO-MQToRedis:689134352 3 Case_Accepted Fault code F1527 taken by maveer APT-ACI-SOLUTIONS2 at 2020-05-

In [11]:
def FindInQueueEvent(CaseEventsList,InterestQueueName=[],shift_start_hour=0,shift_end_hour=6,debug=False):
    
    InterestQueueName = InterestQueueName
    
    #1 Case Number #2 Case Priority #3 Case Qeueue #4 Event Timestamp
    InQueue_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d).*Case_InQueue\s(.+?)\s.*at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    #1 Case Number #2 Case Priority #3 Event Timestamp
    FTS_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d).*FTS_InQueue\s.*at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    UC_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d).*UC_InQueue.*at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    
    CaseInQueueRawList = []
    FTSInQueueRawList = []
    UCInQueueRawList = []
    
    for line in CaseEventsList:
        if 'Case_InQueue' in line:
            #print(line)
            #break
            if re.search(InQueue_Reg,line):
                CaseInQueueRawList.append(re.search(InQueue_Reg,line).group(1)+"-~"+
                              re.search(InQueue_Reg,line).group(2)+"-~"+
                              re.search(InQueue_Reg,line).group(3)+"-~"+
                              re.search(InQueue_Reg,line).group(4))
    
    #CaseInQueueRawList = [re.search(InQueue_Reg,line).group(1)+"-~"+
    #                      re.search(InQueue_Reg,line).group(2)+"-~"+
    #                      re.search(InQueue_Reg,line).group(3)+"-~"+
    #                      re.search(InQueue_Reg,line).group(4) 
    #                      for line in CaseEventsList if 'Case_InQueue' in line]

    for line in CaseEventsList:
        if 'FTS_InQueue' in line:
            #print(line)
            if re.search(FTS_Reg,line):
                #print("Add FTS ".format(line))
                FTSInQueueRawList.append(re.search(FTS_Reg,line).group(1)+"-~"+
                              re.search(FTS_Reg,line).group(2)+"-~FTS-~"+
                              re.search(FTS_Reg,line).group(3))
                
    #FTSInQueueRawList = [re.search(FTS_Reg,line).group(1)+"-~"+
    #                     re.search(FTS_Reg,line).group(2)+"-~FTS-~"+
    #                     re.search(FTS_Reg,line).group(3) 
    #                     for line in CaseEventsList if 'FTS_InQueue' in line]
    
    UCInQueueRawList = [re.search(UC_Reg,line).group(1)+"-~"+
                         re.search(UC_Reg,line).group(2)+"-~UC-~"+
                         re.search(UC_Reg,line).group(3) 
                         for line in CaseEventsList if 'UC_InQueue' in line]
    
    ###### if the case case in before shift start and event in the shift start, it will still be recorded, need to remove ######
    #2020-03-06 01:00:19,511-INFO-MQToRedis:688593987 3 Case_InQueue WW-ACI-Solutions L2NodeAuthPolicy at 2020-03-06 00:58:47
    #Example is as above
    CaseInQueueRawRemoveBeforeShiftEventList = []
    for caseline in CaseInQueueRawList:
        #print(caseline)
        caseno, priority, queue, casetime = caseline.split("-~")
        case_datetime = parser.parse(casetime)
        if case_datetime.hour >= shift_start_hour and case_datetime.hour < shift_end_hour:
            CaseInQueueRawRemoveBeforeShiftEventList.append(caseline)
    CaseInQueueRawList = CaseInQueueRawRemoveBeforeShiftEventList    
    
    if debug:
        print("Original CaseInQueue List length {}".format(len(CaseInQueueRawList)))
        print("Original FTSInQueue List length {}".format(len(FTSInQueueRawList)))
        print("Original UCInQueue List length {}".format(len(UCInQueueRawList)))
        
    NotInterestQueueIndexList = [index for index,case in enumerate(CaseInQueueRawList) if case.split("-~")[2] not in InterestQueueName]
    
    if debug:
        print("Not Interest Queue Index list {}".format(NotInterestQueueIndexList))
    
    CaseInQueueRemoveNotInterestQueueExtList = [line for line in CaseInQueueRawList]
    
    for index in NotInterestQueueIndexList[::-1]:
        #print("Pop not interest {} from list".format(index))
        CaseInQueueRemoveNotInterestQueueExtList.pop(index)
    
    CaseInQueueRemoveNotInterestQueueExtList = list(set(CaseInQueueRemoveNotInterestQueueExtList))
    
    if debug:
        print("After removing NotInterestQueue length {}".format(len(CaseInQueueRemoveNotInterestQueueExtList)))
        #for case in CaseInQueueRemoveNotInterestQueueExtList:
        #    print(case)
    
    DuplicateInQueueIndexList = []
    for index1, raw_case_line1 in enumerate(CaseInQueueRemoveNotInterestQueueExtList):
        caseno1, priority1, queue1, time1 = raw_case_line1.split("-~")
        for index2, raw_case_line2 in enumerate(CaseInQueueRemoveNotInterestQueueExtList[index1+1:]):
            caseno2, priority2, queue2, time2 = raw_case_line2.split("-~")
            #print("Comparing {} {} {} - {} {} {}".format(caseno1,queue1,time1,caseno2,queue2,time2))
            if caseno1 == caseno2 and time1 == time2:
                #print("Found dupliate {} {}".format(index1,index1+index2+1))
                DuplicateInQueueIndexList.append(index1+index2+1)
    if debug:
        print("Duplicate Case Index list {}".format(DuplicateInQueueIndexList))
        for case in DuplicateInQueueIndexList:
            print(case)
            
    CaseInQueueRemoveDupExtList = [line for line in CaseInQueueRemoveNotInterestQueueExtList]
    for index in DuplicateInQueueIndexList[::-1]:
        #print("Pop duplicate {} from list".format(index))
        CaseInQueueRemoveDupExtList.pop(index)
    if debug:
        print("After removing DuplicateCase length {}".format(len(CaseInQueueRemoveDupExtList)))

    return CaseInQueueRemoveDupExtList+FTSInQueueRawList+UCInQueueRawList

InQueueEventList = FindInQueueEvent(allevents_in_date,InterestQueueName,shift_start_hour=0,shift_end_hour=6,debug=False)
InQueueEventList

['689134064-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 03:27:04',
 '689128378-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 04:12:32',
 '689056399-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 03:30:15',
 '689128378-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 04:12:39',
 '689134039-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 02:51:39',
 '689133191-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-21 03:15:03',
 '689124387-~2-~CX-APJC-SYD-ACI-SSPT-~2020-05-21 00:56:31',
 '689134406-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 04:50:06',
 '689133999-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 02:39:45',
 '689128378-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 04:14:19',
 '689134289-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-21 04:08:48',
 '689134216-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 04:21:56',
 '689134086-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-21 03:03:00',
 '689130460-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 01:20:12',
 '689124387-~2-~CX-APJC-SYD-ACI-SSPT-~2020-05-21 00:56:37',
 '689134052-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 02:53:49',
 '689134216-~3-~CX-APJC-BLR-ACI-SSPT-~20

In [15]:
def GetShiftHour(date='2020-04-05'):
    
    date_datetime = parser.parse(date)

    ShiftTime = {
        '2020Summer':['2019-10-17','2020-03-28'],
        '2020InterToWinter':['2020-03-29','2020-04-04'],
        '2020Winter':['2020-04-05','2020-09-01'],
        '2020InterToSummer':['2020-09-02','2020-10-01']
    }
    ShiftHour = {
        'Summer':[1,7],
        'InterToWinter':[0,6],
        'Winter':[0,6],
        'InterToSummer':[1,7]
    }
    
    AESTorAEDT = {
        '10':[
            ['2020-04-04','2020-10-04'],
            ['2021-04-05','2021-10-03']
        ],
        '11':[
            ['2019-10-17','2020-04-04'],
            ['2020-10-05','2021-04-04']
        ]
    }
    
    GMT = -1
    ShiftStartHour = []
    
    for key,period in ShiftTime.items():
        year,sfhit,start_datetime,end_datetime = key[:4],key[4:],parser.parse(period[0]),parser.parse(period[1])
        #print("GetShiftHour - {} v.s. {} {} {} {}".format(date, year,sfhit,period[0],period[1]))
        if date_datetime >= start_datetime and date_datetime <= end_datetime:
            ShiftStartHour = ShiftHour[key[4:]]  #[1, 7]
            break
            
    for gmthour,periodlist in AESTorAEDT.items():
        for period in periodlist:
            #print(period)
            if date_datetime >= parser.parse(period[0]) and date_datetime <= parser.parse(period[1]):
                GMT = gmthour
                break
                
    #print("ShiftStartHourInGMT {} GMT {}".format(ShiftStartHour[0],GMT))
    return ShiftStartHour,int(GMT)

GetShiftHour(date='2020-01-08 08:30:00')


([1, 7], 11)

In [16]:
def isOnShift(timestamp,timezone='America/Los_Angeles',before_shift_hours_offset = 0, after_shift_hours_offset = 0, debug=False):
    
    '''
    before_shift_hours_offset is the hours before shift if would like to be considered to be onshift, default is 0
    after_shift_hours_offset is the hours after shift if would like to be considered to be onshift, default is 0
    before_shift_hours_offset and after_shift_hours_offset is majorly used for dispatched case that dispatched one hour prior to shift would be considered.
    
    local = pytz.timezone ("America/Los_Angeles")
    naive = datetime.datetime.strptime ("2001-2-3 10:11:12", "%Y-%m-%d %H:%M:%S")
    local_dt = local.localize(naive, is_dst=None)
    utc_dt = local_dt.astimezone(pytz.utc)
    India Standard Time (Asia/Kolkata)
    '''
    
    date, time = timestamp.split()
    timezone = timezone.strip()
    
    local = pytz.timezone (timezone)
    local_dt = local.localize(parser.parse(timestamp))
    local_gmt_offset = local_dt.utcoffset().total_seconds()/60/60
    utc_dt = local_dt.astimezone(pytz.utc)
    #print("InputTime:{} TimeZone:{} GMTOffset:{} UTCTime:{}".format(
    #    timestamp,timezone,local_gmt_offset,utc_dt.strftime("%Y-%m-%d %H:%M:%S")))
    utc_date = utc_dt.strftime("%Y-%m-%d")
    utc_time = utc_dt.strftime("%H:%M:%S")
    hour = utc_dt.hour
    minute = utc_dt.minute
    
    shift_hour, GMT_start = GetShiftHour(date)
    orig_shift_hour = shift_hour.copy()
    ###### If offset is considered, plus 24 as event might be at 23:00 - 24:00 the day before ######
    if before_shift_hours_offset != 0 or after_shift_hours_offset != 0:
        shift_hour[0] = shift_hour[0] + 24 - before_shift_hours_offset 
        shift_hour[1] = shift_hour[1] + after_shift_hours_offset + 24
        if hour <= 7:
            hour = hour + 24
            
    if debug:
        print("isOnShift:Shift Hour {} Start GMT {}".format(shift_hour,GMT_start))
        print("isOnShift:Hour {} Minute {}".format(hour,minute))
    
    if hour >= shift_hour[0] and hour < shift_hour[1]:
        if debug:
            print("isOnShift:IsOnShift {}".format(utc_date+' '+utc_time))
        return "On", (utc_dt + datetime.timedelta(hours=(GMT_start+orig_shift_hour[0]))).strftime("%Y-%m-%d %H:%M:%S")
    else:
        if debug:
            print("isOnShift:OffShift {}".format(utc_date+' '+utc_time))
        return "Off", (utc_dt + datetime.timedelta(hours=(GMT_start+orig_shift_hour[0]))).strftime("%Y-%m-%d %H:%M:%S")

timestamp = datetime.datetime.today().strftime("%Y-%m-%d %H:%M:%S")
timestamp = '2020-02-17 16:00:00'
isOnShift(timestamp = '2020-05-16 21:30:00',timezone = 'America/New_York',before_shift_hours_offset=1,debug = False)
isOnShift(timestamp=timestamp,timezone='Australia/Sydney',debug = True)
fd

isOnShift:Shift Hour [1, 7] Start GMT 11
isOnShift:Hour 5 Minute 0
isOnShift:IsOnShift 2020-02-17 05:00:00


('On', '2020-02-17 17:00:00')

In [17]:
###### Find Dispatch Event ######
def FindDispatchEvent(CaseEventsList,debug=False):
    
    RawDispatchedEventList = []
    
    #Group 1.	2932-2941	689101876
    #Group 2.	2942-2943	3
    #Group 3.	2964-3033	APIC Appliance || Maintenance window / Standby (Closed Wednesday 19) 
    #Group 4.	3037-3056	2020-05-17 01:00:00
    #Group 5.	3060-3066	+05:30
    #Group 6.	3067-3102	 India Standard Time (Asia/Kolkata)
    Dispatch_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d).*Case_Dispatched_Out\s(.+?)To\s\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})\(GMT([+-]\d{2}:\d{2})\)(.+)')
    
    for line in CaseEventsList:
        if 'Case_Dispatched_Out' in line: 
            dispatch_line = re.search(Dispatch_Reg,line)
            if dispatch_line:
                caseno = dispatch_line.group(1)
                severity = dispatch_line.group(2)
                casetitle = dispatch_line.group(3).strip()
                timestamp = dispatch_line.group(4)
                #Two type of the timezone
                #2020-02-09 00:30:00(GMT+03:00) Europe/Istanbul
                #2020-01-08 08:30:00(GMT-06:00) Central Standard Time (America/Chicago)
                if re.search(r'.+?\((.+)\)',dispatch_line.group(6)):
                    timezone = re.search(r'.+?\((.+)\)',dispatch_line.group(6)).group(1)
                else:
                    timezone = dispatch_line.group(6)

                ShiftInd , AESTorAEDTtimestamp = ACILib.isOnShift(timestamp,timezone,before_shift_hours_offset=1)
                if ShiftInd == "On":
                    print("OnShift {} {} {} {}".format(caseno,timestamp,timezone,line))
                    RawDispatchedEventList.append("-~".join([caseno,severity,AESTorAEDTtimestamp,casetitle]))

    ###### Remove multiple dispatched and keep only the last one ######
    RemoveDuplicateDispatchedEventIdx = []
    RemoveDuplicateDispatchedEventList = []
    ExistingDispatchCaseDic = {} # {'Date':[CaseNoList]}
    
    for idx,line in enumerate(RawDispatchedEventList[::-1]):
        caseno,casesev,casetime,casetitle = line.split('-~')
        date = parser.parse(casetime).strftime("%Y-%m-%d")
        if date not in ExistingDispatchCaseDic.keys():
            ExistingDispatchCaseDic[date] = []
        if caseno not in ExistingDispatchCaseDic[date]:
            ExistingDispatchCaseDic[date].append(caseno)
        else:
            RemoveDuplicateDispatchedEventIdx.append(len(RawDispatchedEventList)-idx-1)
    
    #print(set(RemoveDuplicateDispatchedEventIdx))
    for idx,line in enumerate(RawDispatchedEventList):
        if idx not in set(RemoveDuplicateDispatchedEventIdx):
            RemoveDuplicateDispatchedEventList.append(line)
    
    ###### Sort the Dispatched Case ######
    DispatchedEventRemainingIdx = [i for i in range(len(RemoveDuplicateDispatchedEventList))]
    SortedDispatchedEventIdx = []
    
    for idx in range(len(RemoveDuplicateDispatchedEventList)):
        smallest_idx = DispatchedEventRemainingIdx[0]
        caseno,casesev,smallest_time,casetitle = RemoveDuplicateDispatchedEventList[smallest_idx].split('-~')
        #print("Round {} Setting Smallest {}".format(smallest_idx,casetime1))
        for idx2,line2 in enumerate(RemoveDuplicateDispatchedEventList):
            if idx2 in SortedDispatchedEventIdx:
                continue
            caseno2,casesev2,casetime2,casetitle2 = line2.split('-~')
            if parser.parse(casetime2) < parser.parse(smallest_time):
                #print("Found Smallest {} {}".format(idx2,casetime2))
                smallest_idx = idx2
                smallest_time = casetime2 
        #print("Round {} Smallext_idx {}".format(idx,smallest_idx))
        DispatchedEventRemainingIdx.remove(smallest_idx)
        SortedDispatchedEventIdx.append(smallest_idx)
    
    SortedDispatchedEventList = []
    for idx in SortedDispatchedEventIdx:
        SortedDispatchedEventList.append(RemoveDuplicateDispatchedEventList[idx])
        
    return SortedDispatchedEventList

RawDispatchedEventList = FindDispatchEvent(allevents,debug=True)
RawDispatchedEventList[:11]

OnShift 688206227 2020-01-10 22:00:00 America/New_York /app/RabbitMQ_TAC_Event_Handler.py-2020-01-08 13:44:52,797-INFO-MQToRedis:688206227 3 Case_Dispatched_Out Requesting Cisco's Assistance for ACI Upgrade To  2020-01-10 22:00:00(GMT-05:00) Eastern Standard Time (America/New_York)
OnShift 688206227 2020-01-17 22:00:00 America/New_York /app/RabbitMQ_TAC_Event_Handler.py-2020-01-08 13:46:45,938-INFO-MQToRedis:688206227 3 Case_Dispatched_Out Requesting Cisco's Assistance for ACI Upgrade To  2020-01-17 22:00:00(GMT-05:00) Eastern Standard Time (America/New_York)
OnShift 687688139 2020-01-10 23:30:00 America/New_York /app/RabbitMQ_TAC_Event_Handler.py-2020-01-10 16:56:53,625-INFO-MQToRedis:687688139 3 Case_Dispatched_Out Assistance with ACI Faults To  2020-01-10 23:30:00(GMT-05:00) Eastern Standard Time (America/New_York)
OnShift 688224481 2020-01-10 18:30:00 America/Los_Angeles /app/RabbitMQ_TAC_Event_Handler.py-2020-01-10 17:25:43,289-INFO-MQToRedis:688224481 3 Case_Dispatched_Out PRO-AC

['688206278-~2-~2020-01-11 13:05:00-~Tetration|| Drive failure',
 '688225292-~3-~2020-01-11 14:15:00-~[GAS NATURAL][Proactive]Legacy to ACI FEX migration (B6)',
 '688224481-~3-~2020-01-11 14:30:00-~PRO-ACTIVE CASE: Tetration Cluster Upgrade',
 "688206227-~3-~2020-01-11 15:00:00-~Requesting Cisco's Assistance for ACI Upgrade",
 '687688139-~3-~2020-01-11 16:30:00-~Assistance with ACI Faults',
 '688237408-~3-~2020-01-15 13:00:00-~Proactive SR - ACI upgrade for EDC (Hospital)',
 '688233158-~3-~2020-01-15 14:00:00-~ACI upgrade',
 '688238897-~3-~2020-01-17 16:00:00-~ACI 3.2(4e) - Proactive SR for upgrade to 4.2(3j)',
 "688206227-~3-~2020-01-18 15:00:00-~Requesting Cisco's Assistance for ACI Upgrade",
 '688277245-~3-~2020-01-21 14:45:00-~Proactive For Telstra Tetration Upgrade from 3.2 to 3.3',
 '688287649-~3-~2020-01-22 13:05:00-~[tetration] About Service Status error']

In [54]:
def SortInQueueEventByTime(CaseInQueueList,date='2020-04-05',debug=True):
    
    CaseInQueueIndexSortByTime = []
    CaseInQueueSortedAndRemoveList = []
    CaseInQueueSortedRemaingList = [i for i in range(len(CaseInQueueList))]
    CaseInQueueRemoveDupExtSortByTimeList = []
    CaseInqueueRemoveDateSortByTimeList = []

    shift_hour,gmt_hour = GetShiftHour(date)
    
    for index1, line1 in enumerate(CaseInQueueList):
        
        smallest_caseno,smallest_priority, smallest_queue,smallest_time = CaseInQueueList[CaseInQueueSortedRemaingList[0]].split("-~")
        smallest_datetime = dateutil.parser.parse(smallest_time)
        CaseInQueueIndexSortByTime.append(CaseInQueueSortedRemaingList[0])
        
        for index2, line2 in enumerate(CaseInQueueList[:]):
            
            if index2 in CaseInQueueSortedAndRemoveList:
                #if debug:
                #    print("Skip {} {}".format(index1,index2))
                continue
            caseno2, priority2, queue2, time2 = line2.split("-~")
            datetime2 = dateutil.parser.parse(time2)
            
            #if debug:
                #print("Compare {} {} - {} {}".format(CaseInQueueSortedRemaingList[0],smallest_datetime,index1+index2+1,datetime2))
            
            if datetime2 < smallest_datetime:
                #if debug:
                    #print("Found smaller index{}".format(index2))
                CaseInQueueIndexSortByTime[index1] = index2
                smallest_datetime = datetime2
                
        #if debug:
        #    print("Iteration {} pop {}".format(index1,CaseInQueueIndexSortByTime[index1]))
        CaseInQueueSortedAndRemoveList.append(CaseInQueueIndexSortByTime[index1])
        CaseInQueueSortedRemaingList.remove(CaseInQueueIndexSortByTime[index1])
        
        #if debug:
        #    print(CaseInQueueIndexSortByTime)
        #    print(CaseInQueueSortedRemaingList)
        #    print("Sorted Index from CaseInQueueList {}".format(CaseInQueueIndexSortByTime))

    for index in CaseInQueueIndexSortByTime:
        CaseInQueueRemoveDupExtSortByTimeList.append(CaseInQueueList[index])
    
    if debug:
        print("After Revming Dup Case - to be remoeved DUP FTS/UC")
        for caseline in CaseInQueueRemoveDupExtSortByTimeList:
            print(caseline)
    ###### Remove Dup FTS  List and only keep the last one ######
    ###### Remove Dup UC  List and only keep the last one ######
    ###### Remove Dup Case within 20 seconds and 2 event next to each other and only keep the last one ######
    InQueueFTSList = [] #Index from CaseInQueueRemoveDupExtSortByTimeList #Case , #Timestamp
    InQueueDupFTSIndexList = []
    InQueueUCList = [] #Index from CaseInQueueRemoveDupExtSortByTimeList #Case , #Timestamp
    InQueueDupUCIndexList = []

    LastCaseList = [] #Index from CaseInQueueRemoveDupExtSortByTimeList #Case #Queue #Timestamp
    InQueueDupCaseIndexList = []
    
    ######## Dup case condidition < 1 minutes
    for index, caseline in enumerate(CaseInQueueRemoveDupExtSortByTimeList):
        caseno, priority, queue, casetime = caseline.split("-~")
        if queue == 'FTS':
            InQueueFTSList.append([index,caseno,casetime])
        elif queue == 'UC':
            InQueueUCList.append([index,caseno,casetime])
        else:
            for LastCase in LastCaseList:
                if LastCase and LastCase[1] == caseno and LastCase[3] == queue:
                    case_datetime = parser.parse(casetime)
                    last_datetime = parser.parse(LastCase[4])
                    ######## Dup case condition ######
                    #1. same events come in < 60 seconds, 2 events in a row 
                    #2. priority are different , 2 events in a row
                    if (case_datetime - last_datetime).seconds <= 60 or LastCase[2] != priority:
                        if debug:
                            print("Found Dup Case {} {} {}".format(LastCase[0],caseno,LastCase[4]))
                        InQueueDupCaseIndexList.append(LastCase[0])
            LastCaseList.append([index,caseno,priority,queue,casetime])
    InQueueDupCaseIndexList = set(InQueueDupCaseIndexList)
    
    ###### Get InQueueDupFTSIndexList in order ######
    for index1, fts_case1 in enumerate(InQueueFTSList):
        for index2, fts_case2 in enumerate(InQueueFTSList[index1+1:]):
            if fts_case1[1] == fts_case2[1]:
                if debug:
                    print("Dup FTS index {} with {} and remove {}".format(fts_case1[0],fts_case2[0],fts_case1[0]))
                InQueueDupFTSIndexList.append(fts_case1[0])
    ###### Get InQueueDupUCIndexList in order ######
    for index1, uc_case1 in enumerate(InQueueUCList):
        for index2, uc_case2 in enumerate(InQueueUCList[index1+1:]):
            if uc_case1[1] == uc_case2[1]:
                if debug:
                    print("Dup UC index {} with {} and remove {}".format(uc_case1[0],uc_case2[0],uc_case1[0]))
                InQueueDupUCIndexList.append(uc_case1[0])
    if debug:
        print("Dup Case List {}".format(" ".join([str(index) for index in InQueueDupCaseIndexList])))
        print("Dup FTS Case List {}".format(" ".join([str(index) for index in set(InQueueDupFTSIndexList)])))
        print("Dup UC Case List {}".format(" ".join([str(index) for index in set(InQueueDupUCIndexList)])))
        
    InQueueDupIndexList = sorted([*InQueueDupCaseIndexList,*set(InQueueDupFTSIndexList),*set(InQueueDupUCIndexList)])
    
    for index in InQueueDupIndexList[::-1]:
        CaseInQueueRemoveDupExtSortByTimeList.pop(index)
        
    ###### Remove Case which is not for that date ######
    for caseline in CaseInQueueRemoveDupExtSortByTimeList:
        caseno, priority, queue, time = caseline.split("-~")
        line_datetime = parser.parse(time)
        if line_datetime.strftime("%Y-%m-%d") == date:
            CaseInqueueRemoveDateSortByTimeList.append(caseline)

    return CaseInqueueRemoveDateSortByTimeList

InQueueEventInOrderList = SortInQueueEventByTime(InQueueEventList,date=date,debug=True)
InQueueEventInOrderList

After Revming Dup Case - to be remoeved DUP FTS/UC
689115309-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 00:02:20
689115219-~3-~FTS-~2020-05-19 00:04:16
689103538-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 00:19:11
689103538-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 00:19:28
688674634-~3-~FTS-~2020-05-19 00:20:42
689056202-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 00:25:09
689056202-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 00:25:18
689056202-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 00:25:21
689115032-~3-~FTS-~2020-05-19 00:54:47
689115458-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 00:56:51
689115032-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 01:21:37
689115032-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 01:21:44
688924342-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 01:29:49
688924342-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 01:29:56
689115289-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 02:00:22
689115706-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 02:11:51
689115776-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 02:27:42
689115921-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 0

['689115309-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 00:02:20',
 '689115219-~3-~FTS-~2020-05-19 00:04:16',
 '689103538-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 00:19:28',
 '688674634-~3-~FTS-~2020-05-19 00:20:42',
 '689056202-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 00:25:21',
 '689115032-~3-~FTS-~2020-05-19 00:54:47',
 '689115458-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 00:56:51',
 '689115032-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 01:21:44',
 '688924342-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 01:29:56',
 '689115289-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 02:00:22',
 '689115706-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 02:11:51',
 '689115776-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 02:27:42',
 '689115921-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 03:05:49',
 '689116106-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-19 04:03:36',
 '688451023-~3-~UC-~2020-05-19 04:03:45',
 '689098491-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-19 04:10:42']

In [55]:
CaseStatsDic = {"SydQueueVol":0,"BGLQueueVol":0,
            "SydTakenVol":0,"BGLTakenVol":0,
            'FTSCaseVol':0,'UCCaseVol':0,
            "SydQueueVolByPri":[0,0,0,0,0,0],"BGLQueueVolByPri":[0,0,0,0,0,0], #0 FTS #1,2,3,4 Priority #5 Urgent Collab
            "SydTakenVolByPri":[0,0,0,0,0,0],"BGLTakenVolByPri":[0,0,0,0,0,0], #0 FTS #1,2,3,4 Priority #5 Urgent Collab
            "SydOnShiftEngr":1,"BGLOnShiftEngr":1,
            "SydOnShiftEngrTakenVol":0,"BGLOnShiftEngrTakenVol":1,
            "SYDHelper":0,"BGLHelper":0,
            }

def GetStatsFromInQueueEventInOrderList(InQueueEventInOrderList,CaseStatsDic):
    
    ###### Getting partial CaseStatsDic Stats from the orderred List ######
    for caseline in InQueueEventInOrderList:
        caseno, priority, queue, time = caseline.split("-~")
        if 'SYD' in queue:
            CaseStatsDic['SydQueueVol'] = CaseStatsDic['SydQueueVol'] + 1
            CaseStatsDic['SydQueueVolByPri'][int(priority)] = CaseStatsDic['SydQueueVolByPri'][int(priority)]+1
        elif 'BLR' in queue:
            CaseStatsDic['BGLQueueVol'] = CaseStatsDic['BGLQueueVol'] + 1
            CaseStatsDic['BGLQueueVolByPri'][int(priority)] = CaseStatsDic['BGLQueueVolByPri'][int(priority)]+1
        elif 'FTS' in queue:
            CaseStatsDic['FTSCaseVol'] = CaseStatsDic['FTSCaseVol'] + 1
        elif 'UC' in queue:
            CaseStatsDic['UCCaseVol'] = CaseStatsDic['UCCaseVol'] + 1
    
    return CaseStatsDic

CaseStatsDic = GetStatsFromInQueueEventInOrderList(InQueueEventInOrderList,CaseStatsDic)
CaseStatsDic

{'SydQueueVol': 5,
 'BGLQueueVol': 7,
 'SydTakenVol': 0,
 'BGLTakenVol': 0,
 'FTSCaseVol': 3,
 'UCCaseVol': 1,
 'SydQueueVolByPri': [0, 0, 0, 5, 0, 0],
 'BGLQueueVolByPri': [0, 0, 0, 7, 0, 0],
 'SydTakenVolByPri': [0, 0, 0, 0, 0, 0],
 'BGLTakenVolByPri': [0, 0, 0, 0, 0, 0],
 'SydOnShiftEngr': 1,
 'BGLOnShiftEngr': 1,
 'SydOnShiftEngrTakenVol': 0,
 'BGLOnShiftEngrTakenVol': 1,
 'SYDHelper': 0,
 'BGLHelper': 0}

In [18]:
#/app/RabbitMQ_TAC_Event_Handler.py-2020-04-18 11:23:26,480-INFO-MQToRedis:688881426 3 Case_Accepted Chileaf402 -High SSD usage observed- Equipment Flash Warning taken by dashmari HTTS-EMEAR-DC-ACI at 2020-04-18 11:21:24

def CaseAcceptInOrderByTime(AllEventsList,date='2020-02-27',debug=False):

    shift_hour,gmt_hour = GetShiftHour(date)
    
    #0 Severity #1 CaseAccept Type Case|FTS|UC #2 ccoid #3 Workgroup #4 Time
    Accept_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d)\s(Case|FTS|UC)_Accepted.*\sby\s(\w+)\s(.*)\s{0,1}at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    
    New_AllEventsList = []
    for event in AllEventsList:
        if 'Exception' not in event:
            New_AllEventsList.append(event)
            
    CaseRawAcceptList = []
    for line in AllEventsList:
        if re.search(r'_Accepted',line):
            #print(line)
            result = re.search(Accept_Reg,line)
            workgroup = result.group(5).strip()
            if not workgroup :
                workgroup = "Unknown"
            elif 'FTS_Accepted' in line:
                print(line)
                workgroup = "FTS_WORKGROUP"
            elif 'UC_Accepted' in line:
                print(line)
                workgroup = "UC_WORKGROUP"
                
            CaseRawAcceptList.append(result.group(1)+"-~"+result.group(2)+"-~"+result.group(4)+"-~"+
                                    workgroup+"-~"+result.group(6))
            
    #CaseRawAcceptList = [re.search(Accept_Reg,line).group(1)+"-~"+
    #                     re.search(Accept_Reg,line).group(2)+"-~"+
    #                     re.search(Accept_Reg,line).group(4)+"-~"+
    #                     re.search(Accept_Reg,line).group(5)+"-~"+
    #                     re.search(Accept_Reg,line).group(6)
    #                     for line in New_AllEventsList if re.search(r'_Accepted',line)]
    
    if debug:
        print("CasAcceptInOrderByTime:Raw Accept List Length {}".format(len(CaseRawAcceptList)))
    
    DuplicateAcceptIndexList = []
    for index1, raw_case_line1 in enumerate(CaseRawAcceptList):
        caseno1, severity1, queue1, workgroup, time1 = raw_case_line1.split("-~")
        for index2, raw_case_line2 in enumerate(CaseRawAcceptList[index1+1:]):
            caseno2, severity2, queue2, workgroup, time2 = raw_case_line2.split("-~")
            if debug:
                pass
                #print("CasAcceptInOrderByTime:Comparing {} {} {} - {} {} {}".format(caseno1,queue1,time1,caseno2,queue2,time2))
            if caseno1 == caseno2 and time1 == time2:
                if debug:
                    print("Found dupliate {} {}".format(index1,index1+index2+1))
                DuplicateAcceptIndexList.append(index1+index2+1)
    DuplicateAcceptIndexList = sorted(list(set(DuplicateAcceptIndexList)))
    if debug:
        print("CasAcceptInOrderByTime:Duplicate Case Index list {} {}".format(len(DuplicateAcceptIndexList),DuplicateAcceptIndexList))
    
    CaseAcceptRemoveDupExtList = [line for line in CaseRawAcceptList]
    for index in DuplicateAcceptIndexList[::-1]:
        if debug:
            print("CasAcceptInOrderByTime:Pop duplicate {} from list".format(index))
        CaseAcceptRemoveDupExtList.pop(index)
    if debug:
        print("CasAcceptInOrderByTime:After removing DuplicateCase length {}".format(len(CaseAcceptRemoveDupExtList)))

    ###### Remove the case Accept time out of the current date ######
    CaseAcceptRemoveDupExtListWithDate = []
    for caseline in CaseAcceptRemoveDupExtList:
        caseno, severity, ccoid, workgroup, casetime = caseline.split("-~")
        line_datetime = parser.parse(casetime)
        if line_datetime.strftime("%Y-%m-%d") == date:
            CaseAcceptRemoveDupExtListWithDate.append(caseline)

    UniqueAcceptEventInOrderList = []
    for line in CaseAcceptRemoveDupExtListWithDate[::-1]:
        caseno, severity, ccoid, workgroup, timestamp = line.split('-~')
        isFoundDupCase = False
        for uniline in UniqueAcceptEventInOrderList[::-1]:
            unicaseno, severity, uniccoid, workgroup, unitimestamp = uniline.split('-~')
            if (unicaseno == caseno) and (uniccoid == ccoid):
                isFoundDupCase = True
                break
        if not isFoundDupCase:
            UniqueAcceptEventInOrderList.append(line) 
            
    return UniqueAcceptEventInOrderList[::-1]

AcceptEventInOrderList = CaseAcceptInOrderByTime(allevents_in_date,date=date,debug=False)
AcceptEventInOrderList

RabbitMQ_ACI_Event.log-2020-05-21 00:27:53,498-INFO-MQToRedis:689114061 3 UC_Accepted Verify network link output/input errors on Nexus 6K taken by gauvasud IND-DCSW at 2020-05-21 00:25:09
RabbitMQ_ACI_Event.log-2020-05-21 00:37:51,253-INFO-MQToRedis:689133456 2 UC_Accepted C1-N9K-C93180YC-EX// endpoint learning taken by tonzeng US-ACI-SOL-WEST at 2020-05-21 00:34:54
RabbitMQ_ACI_Event.log-2020-05-21 00:47:57,217-INFO-MQToRedis:688534492 3 UC_Accepted Cisco ACI - external switch MAC flapping taken by maveer US-ACI-SOL-WEST at 2020-05-21 00:45:29
RabbitMQ_ACI_Event.log-2020-05-21 01:05:17,694-INFO-MQToRedis:689132928 2 FTS_Accepted APIC-2 IS DOWN taken by maveer US-ACI-SOL-WEST at 2020-05-21 01:01:57
RabbitMQ_ACI_Event.log-2020-05-21 01:19:40,618-INFO-MQToRedis:689133456 2 UC_Accepted C1-N9K-C93180YC-EX// endpoint learning taken by tonzeng APT-ACI-SOLUTIONS at 2020-05-21 01:15:57
RabbitMQ_ACI_Event.log-2020-05-21 01:22:46,035-INFO-MQToRedis:689133456 2 UC_Accepted C1-N9K-C93180YC-EX// en

['689133456-~2-~mislasar-~US-ACI-SOL-WEST-~2020-05-21 00:09:54',
 '689126972-~3-~tskanai-~JAPAN-ACI-~2020-05-21 00:16:22',
 '689114061-~3-~gauvasud-~UC_WORKGROUP-~2020-05-21 00:25:09',
 '688534492-~3-~maveer-~UC_WORKGROUP-~2020-05-21 00:45:29',
 '689124387-~2-~siddhp-~APT-ACI-SOLUTIONS-~2020-05-21 01:11:43',
 '689133456-~2-~tonzeng-~APT-ACI-SOLUTIONS-~2020-05-21 01:15:57',
 '689130460-~2-~reperuma-~APT-ACI-SOLUTIONS2-~2020-05-21 01:25:28',
 '689133555-~3-~piguo-~APJ-CHINA-GCC-~2020-05-21 01:41:56',
 '689133894-~3-~dayamamo-~JAPAN-NEXUS-~2020-05-21 02:15:49',
 '689133924-~3-~lowtouch-~VIRTUAL-~2020-05-21 02:19:53',
 '689133555-~3-~dawang-~APT-GC-Solution-~2020-05-21 02:21:42',
 '689133894-~3-~ykawano-~JAPAN-HTTS-FLETS-~2020-05-21 02:35:19',
 '689133999-~2-~kahande-~APT-ACI-SOLUTIONS2-~2020-05-21 02:45:58',
 '689133923-~3-~pli3-~APT-GC-Solution-~2020-05-21 02:47:05',
 '689134015-~3-~lchuan-~APT-GC-Solution-~2020-05-21 02:58:00',
 '689134064-~3-~lowtouch-~VIRTUAL-~2020-05-21 02:57:20',
 '

In [1]:
SYDEngrList = ['tonzeng','minkwong','wilchong','debabbar','lindawa','junwa']
SYDOtherEngrList = ['zdazhi','zmeng']
BGLEngrList = ['deepaky','knagavol','jawalia','prpratee','maveer','raghb','roagraw2','shparanj','vkalmath']
BGLOtherEngrList = ['vnandan','anirukas','reperuma','deepakba','bharatkc','kdoodi']

def UpdateEngrListFromAcceptEventInOrderList(AcceptEventInOrderList,EngrList,WKGroupList=['APT-ACI-SOLUTIONS','APT-ACI-SOLUTIONS2']):
    
    SYDEngrList = EngrList[0]
    SYDOtherEngrList = EngrList[1]
    BGLEngrList = EngrList[2]
    BGLOtherEngrList = EngrList[3]
    
    Syd_group_Nname = WKGroupList[0]
    BLR_group_name =WKGroupList[1]
    
    for caseline in AcceptEventInOrderList:
        caseno, severity, ccoid, workgroup, time = caseline.split("-~")
        if workgroup == 'UC_WORKGROUP' and ccoid in SYDEngrList:
            workgroup = 'APT-ACI-SOLUTIONS'
        elif ccoid in BGLEngrList:
            workgroup = 'APT-ACI-SOLUTIONS2'
        if workgroup == "Unknown":
            continue
        if workgroup == Syd_group_Nname:
            if ccoid not in SYDEngrList:
                SYDEngrList.append(ccoid)
                print('Updating {} in SydEngrList'.format(ccoid))
        elif workgroup == BLR_group_name:
            if ccoid not in BGLEngrList:
                print('Updating {} in BGLEngrList'.format(ccoid))
                BGLEngrList.append(ccoid)
                if ccoid in BGLOtherEngrList:
                    print('Removing {} from BGLOtherEngrList'.format(ccoid))
                    BGLOtherEngrList.remove(ccoid)
            elif ccoid in BGLOtherEngrList:
                print('Updating {} in BGLOtherEngrList'.format(ccoid))
                if ccoid in BGLEngrList:
                    print('Removing {} from BGLEngrList'.format(ccoid))
                    BGLEngrList.remove(ccoid)
                    
    return [SYDEngrList,SYDOtherEngrList,BGLEngrList,BGLOtherEngrList]
                    
SYDEngrList,SYDOtherEngrList,BGLEngrList,BGLOtherEngrList = UpdateEngrListFromAcceptEventInOrderList(AcceptEventInOrderList,[SYDEngrList,SYDOtherEngrList,BGLEngrList,BGLOtherEngrList])

SYDEngrList,SYDOtherEngrList,BGLEngrList, BGLOtherEngrList

NameError: name 'AcceptEventInOrderList' is not defined

In [20]:
###### Case Accept Stats ######
def GetStatsFromCaseAcceptInOrderByTime(AcceptEventInOrderList,CaseStatsDic,EngrList):
    
    SYDEngrList = EngrList[0]
    SYDOtherEngrList = EngrList[1]
    BGLEngrList = EngrList[2]
    BGLOtherEngrList = EngrList[3]
        
    SydOnShiftEngrList = []
    SydOtherOnshiftEngrList = []

    BGLOnShiftEngrList = []
    BGLOtherOnshiftEngrList = []
    
    ###### Getting partial CaseStatsDic Stats from the orderred List ######
    for caseline in AcceptEventInOrderList:
        caseno, severity, ccoid, workgroup, time = caseline.split("-~")
        if ccoid in SYDEngrList:
            if ccoid not in SydOnShiftEngrList:
                SydOnShiftEngrList.append(ccoid)
        elif ccoid in SYDOtherEngrList:
            if ccoid not in SydOtherOnshiftEngrList:
                SydOtherOnshiftEngrList.append(ccoid)
        elif ccoid in BGLEngrList:
            if ccoid not in BGLOnShiftEngrList:
                BGLOnShiftEngrList.append(ccoid)
        elif ccoid in BGLOtherEngrList:
            if ccoid not in BGLOtherOnshiftEngrList:
                BGLOtherOnshiftEngrList.append(ccoid)
                
    CaseStatsDic['SydOnShiftEngr'] = SydOnShiftEngrList
    CaseStatsDic['BGLOnShiftEngr'] = BGLOnShiftEngrList
    CaseStatsDic['SYDHelper'] = SydOtherOnshiftEngrList
    CaseStatsDic['BGLHelper'] = BGLOtherOnshiftEngrList
    
    return CaseStatsDic

CaseStatsDic = GetStatsFromCaseAcceptInOrderByTime(AcceptEventInOrderList,CaseStatsDic,[SYDEngrList,SYDOtherEngrList,BGLEngrList,BGLOtherEngrList])
CaseStatsDic

{'SydQueueVol': 0,
 'BGLQueueVol': 0,
 'SydTakenVol': 0,
 'BGLTakenVol': 0,
 'FTSCaseVol': 0,
 'UCCaseVol': 0,
 'SydQueueVolByPri': [0, 0, 0, 0, 0, 0],
 'BGLQueueVolByPri': [0, 0, 0, 0, 0, 0],
 'SydTakenVolByPri': [0, 0, 0, 0, 0, 0],
 'BGLTakenVolByPri': [0, 0, 0, 0, 0, 0],
 'SydOnShiftEngr': ['siddhp', 'tonzeng', 'wilchong', 'junwa', 'debabbar'],
 'BGLOnShiftEngr': ['maveer',
  'reperuma',
  'kahande',
  'raghb',
  'prpratee',
  'deepaky',
  'jawalia',
  'hethakur'],
 'SydOnShiftEngrTakenVol': 0,
 'BGLOnShiftEngrTakenVol': 1,
 'SYDHelper': ['zmeng'],
 'BGLHelper': []}

In [21]:
CaseTakenByEngrDic = {}

for case in AcceptEventInOrderList:
    
    caseno,severity,ccoid,workgroup,time = case.split("-~")
    if ccoid not in CaseTakenDic.keys():
        CaseTakenDic[ccoid] = []
    if caseno not in CaseTakenDic[ccoid]:
        CaseTakenDic[ccoid].append(caseno)
    if ccoid not in OnShiftEngrList and (ccoid in SYDEngrList or ccoid in BGLEngrList 
                                         or ccoid in BGLOtherEngrList or ccoid in SYDOtherEngrList):
        OnShiftEngrList.append(ccoid)
    if caseno not in CaseTakenByEngrDic.keys():
        CaseTakenByEngrDic[caseno] = []
        
    if ccoid in SYDEngrList or ccoid in SYDOtherEngrList:
        CaseStatsDic['SydTakenVolByPri'][int(severity)] = CaseStatsDic['SydTakenVolByPri'][int(severity)] + 1
        CaseTakenByEngrDic[caseno].append(ccoid+"(SYD)")
    elif ccoid in BGLEngrList or ccoid in BGLOtherEngrList:
        CaseStatsDic['BGLTakenVolByPri'][int(severity)] = CaseStatsDic['BGLTakenVolByPri'][int(severity)] + 1
        CaseTakenByEngrDic[caseno].append(ccoid+"(BLR)")
    elif ccoid == 'lowtouch':
        CaseTakenByEngrDic[caseno].append(ccoid+"(HW)")
    else:
        CaseTakenByEngrDic[caseno].append(ccoid+"(Unknown)")
        
CaseTakenByEngrDic,CaseStatsDic

({'689133456': ['mislasar(Unknown)', 'tonzeng(SYD)'],
  '689126972': ['tskanai(Unknown)'],
  '689114061': ['gauvasud(Unknown)'],
  '688534492': ['maveer(BLR)'],
  '689124387': ['siddhp(SYD)'],
  '689130460': ['reperuma(BLR)'],
  '689133555': ['piguo(Unknown)', 'dawang(Unknown)'],
  '689133894': ['dayamamo(Unknown)', 'ykawano(Unknown)'],
  '689133924': ['lowtouch(HW)', 'jiangygu(Unknown)'],
  '689133999': ['kahande(BLR)'],
  '689133923': ['pli3(Unknown)'],
  '689134015': ['lchuan(Unknown)'],
  '689134064': ['lowtouch(HW)', 'deepaky(BLR)'],
  '689134052': ['raghb(BLR)'],
  '689134039': ['prpratee(BLR)'],
  '689134086': ['wilchong(SYD)'],
  '689132928': ['maveer(BLR)'],
  '689133191': ['junwa(SYD)'],
  '689056399': ['jawalia(BLR)'],
  '689057434': ['zmeng(SYD)'],
  '689134171': ['yuxuliu(Unknown)'],
  '689134289': ['debabbar(SYD)'],
  '689128378': ['hethakur(BLR)'],
  '689134216': ['raghb(BLR)'],
  '689134352': ['maveer(BLR)'],
  '689134406': ['deepaky(BLR)']},
 {'SydQueueVol': 0,
  'BGLQ

In [22]:
for ccoid, caselist in CaseTakenDic.items():
    if ccoid in SYDEngrList:
        CaseTakenDicInclNo['SYD'][ccoid] = list([len(caselist),*caselist])
    elif ccoid in SYDOtherEngrList:
        CaseTakenDicInclNo['SYD_Other'][ccoid] = list([len(caselist),*caselist])
    elif ccoid in BGLEngrList:
        CaseTakenDicInclNo['BGL'][ccoid] = list([len(caselist),*caselist])
    elif ccoid in BGLOtherEngrList:
        CaseTakenDicInclNo['BGL_Other'][ccoid] = list([len(caselist),*caselist])
CaseTakenDicInclNo

{'SYD': {'siddhp': [1, '689124387'],
  'tonzeng': [1, '689133456'],
  'wilchong': [1, '689134086'],
  'junwa': [1, '689133191'],
  'debabbar': [1, '689134289']},
 'SYD_Other': {'zmeng': [1, '689057434']},
 'BGL': {'maveer': [3, '688534492', '689132928', '689134352'],
  'reperuma': [1, '689130460'],
  'kahande': [1, '689133999'],
  'raghb': [2, '689134052', '689134216'],
  'prpratee': [1, '689134039'],
  'deepaky': [2, '689134064', '689134406'],
  'jawalia': [1, '689056399'],
  'hethakur': [1, '689128378']},
 'BGL_Other': {}}

In [25]:
shift_hour,gmt_hour = ACILib.GetShiftHour(date)
print(date)
print(shift_hour)
print(gmt_hour)

InQueueEventInOrderListWithEngr = []
for caseline in InQueueEventInOrderList:
    caseno,priority,queue,timestamp = caseline.split("-~")
    ###### CaseStatsDic FTS Stats ######
    if queue == 'FTS':
        for ccoid,caselist in CaseTakenDic.items():
            if caseno in caselist:
                if ccoid in SYDEngrList or ccoid in SYDOtherEngrList:
                    CaseStatsDic['SydTakenVolByPri'][0] = CaseStatsDic['SydTakenVolByPri'][0] + 1
                    #CaseStatsDic['SydTakenVol'] = CaseStatsDic['SydTakenVol'] + 1
                elif ccoid in BGLEngrList or ccoid in BGLOtherEngrList:
                    CaseStatsDic['BGLTakenVolByPri'][0] = CaseStatsDic['BGLTakenVolByPri'][0] + 1
                    #CaseStatsDic['BGLTakenVol'] = CaseStatsDic['BGLTakenVol'] + 1
    
    ###### CaseStatsDic UC Stats ######
    elif queue == 'UC':
        for ccoid,caselist in CaseTakenDic.items():
            if caseno in caselist:
                if ccoid in SYDEngrList or ccoid in SYDOtherEngrList:
                    CaseStatsDic['SydTakenVolByPri'][5] = CaseStatsDic['SydTakenVolByPri'][5] + 1
                    #CaseStatsDic['SydTakenVol'] = CaseStatsDic['SydTakenVol'] + 1
                elif ccoid in BGLEngrList or ccoid in BGLOtherEngrList:
                    CaseStatsDic['BGLTakenVolByPri'][5] = CaseStatsDic['BGLTakenVolByPri'][5] + 1   
                    #CaseStatsDic['BGLTakenVol'] = CaseStatsDic['BGLTakenVol'] + 1
    
    if caseno in CaseTakenByEngrDic.keys():
        InQueueEventInOrderListWithEngr.append(caseno+'-~'+priority+"-~"+queue+'-~'+
                                               (parser.parse(timestamp)+datetime.timedelta(hours=(gmt_hour+shift_hour[0]))).strftime("%Y-%m-%d %H:%M:%S")+
                                               '-~'+"-~".join(CaseTakenByEngrDic[caseno]))
    else:
        InQueueEventInOrderListWithEngr.append(caseno+'-~'+priority+"-~"+queue+'-~'+
                                               (parser.parse(timestamp)+datetime.timedelta(hours=(gmt_hour+shift_hour[0]))).strftime("%Y-%m-%d %H:%M:%S")
                                               )
InQueueEventInOrderListWithEngr

2020-05-21
[0, 6]
10


NameError: name 'InQueueEventInOrderList' is not defined

In [24]:
import glob
###### InQueue file parsing Read the current Parsing file 2020_InQueue.txt ######
jupyter_container_path = '/home/jovyan'
flask_container_path = ''
current_container_path = jupyter_container_path
eventfilenames = current_container_path + '/HTTSDashboard/logs/ACI/events/*_InQueueEvent.txt'
eventfilelist = glob.glob(eventfilenames)
for eventfile in eventfilelist:
    print(eventfile)
    with open(eventfile,'r') as f:
        InQueueEvents = f.readlines()
    f.close()
InQueueEvents = [event.strip() for event in InQueueEvents]
for index, event in enumerate(InQueueEvents):
    if index == 1092:
        print(event)
InQueueEvents[-20:]

/home/jovyan/HTTSDashboard/logs/ACI/events/2020_InQueueEvent.txt
688951489-~3-~CX-APJC-BLR-ACI-SSPT-~2020-04-25 05:30:08


['689121083-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-20 04:46:35',
 '689125571-~1-~CX-APJC-SYD-ACI-SSPT-~2020-05-20 04:49:52',
 '689125640-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-20 05:12:53',
 '689133456-~2-~UC-~2020-05-21 00:31:31',
 '688534492-~3-~FTS-~2020-05-21 00:40:38',
 '689132928-~2-~FTS-~2020-05-21 00:49:22',
 '689124387-~2-~CX-APJC-SYD-ACI-SSPT-~2020-05-21 00:56:37',
 '689130460-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 01:22:04',
 '689133999-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 02:39:45',
 '689134039-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 02:51:39',
 '689134052-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 02:53:49',
 '689134086-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-21 03:03:00',
 '689133191-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-21 03:15:09',
 '689134064-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 03:27:04',
 '689056399-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 03:30:21',
 '689134289-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-21 04:08:48',
 '689128378-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-21 04:14:19',
 '689134216-~3-~CX-APJC-BLR-ACI-

In [6]:
######Function to remove a speicifc date inqueue events ######
def RemoveInQueueEventByDate(InQueueEvents,date):
    ######Need to deep copy the list ######
    RemovedDateInQueueEvents = [event for event in InQueueEvents]
    for index,event in enumerate(InQueueEvents[::-1]):
        caseno,severity,workgroup,casetime = event.split('-~')
        if parser.parse(casetime).strftime("%Y-%m-%d") == date:
            RemovedDateInQueueEvents.pop(len(InQueueEvents)-index-1)
        if parser.parse(casetime) < parser.parse(date):
            break
    return RemovedDateInQueueEvents
date = (datetime.datetime.today() - datetime.timedelta(days=0)).strftime("%Y-%m-%d")
RemovedDateInQueueEvents = RemoveEventByDate(InQueueEvents,date)
RemovedDateInQueueEvents[-10:]

NameError: name 'RemoveEventByDate' is not defined

In [5]:
AddEventList = [
    '688967055-~3-~WW-ACI-Solutions-~2020-04-27 02:03:59',
    '688913179-~3-~CX-APJC-BLR-ACI-SSPT-~2020-04-27 03:55:39'
]

######Function to add a speicifc date inqueue inorder events ######
def AddInQueueEventByDate(InQueueEvents,AddList,date):
    ######Need to deep copy the list ######
    AddedDateInQueueEvents = [event for event in InQueueEvents]
    insert_index = len(InQueueEvents)
    for index,event in enumerate(InQueueEvents):
        caseno,severity,workgroup,casetime = event.split('-~')
        if parser.parse(casetime) > parser.parse(date):
            insert_index = index
            break
    print("Insert Index {}".format(insert_index))
    for event in AddList[::-1]:
        print("Inderting {} at {}".format(event,insert_index))
        AddedDateInQueueEvents.insert(insert_index,event)
    return AddedDateInQueueEvents

date = (datetime.datetime.today() - datetime.timedelta(days=0)).strftime("%Y-%m-%d")
AddEventByDate = AddInQueueEventByDate(InQueueEvents,AddEventList,date)
AddEventByDate[-25:]

NameError: name 'InQueueEvents' is not defined

In [257]:
import glob
###### Accept file parsing Read the current Parsing file 2020_InQueue.txt ######
jupyter_container_path = '/home/jovyan'
flask_container_path = ''
current_container_path = jupyter_container_path
eventfilenames = current_container_path + '/HTTSDashboard/logs/ACI/events/*_AcceptEvent.txt'
eventfilelist = glob.glob(eventfilenames)
for eventfile in eventfilelist:
    print(eventfile)
    with open(eventfile,'r') as f:
        AcceptEvents = f.readlines()
    f.close()
AcceptEvents = [event.strip() for event in AcceptEvents]
AcceptEvents[-20:]

/home/jovyan/HTTSDashboard/logs/ACI/events/2020_AcceptEvent.txt


['689047162-~3-~minkwong-~2020-05-08 03:36:54',
 '689047345-~4-~jawalia-~2020-05-08 03:53:12',
 '689042734-~3-~tonzeng-~2020-05-08 04:08:53',
 '688856354-~3-~gauvasud-~2020-05-08 04:13:33',
 '688771603-~3-~gauvasud-~2020-05-08 04:14:56',
 '688889442-~3-~deepaky-~2020-05-08 04:26:32',
 '689047453-~3-~hethakur-~2020-05-08 04:28:46',
 '689047486-~3-~maveer-~2020-05-08 04:37:49',
 '689047359-~3-~lchuan-~2020-05-08 04:40:50',
 '688608334-~3-~kahande-~2020-05-08 04:49:28',
 '688921797-~3-~kahande-~2020-05-08 04:48:48',
 '689017662-~3-~minkwong-~2020-05-08 05:11:59',
 '688938176-~3-~kahande-~2020-05-08 05:24:50',
 '688822797-~3-~knagavol-~2020-05-08 05:27:03',
 '689047630-~2-~raghb-~2020-05-08 05:30:25',
 '688938176-~3-~reperuma-~2020-05-08 05:32:12',
 '689047664-~3-~debabbar-~2020-05-08 05:38:12',
 '689047657-~3-~jiangygu-~2020-05-08 05:44:33',
 '688995998-~1-~debabbar-~2020-05-08 05:55:57',
 '689047706-~3-~siddhp-~2020-05-08 05:56:31']

In [258]:
######Function to remove a speicifc date accept events ######
def RemoveAcceptEventByDate(AcceptEvents,date):
    ######Need to deep copy the list ######
    RemovedDateAcceptEvents = [event for event in AcceptEvents]
    for index,event in enumerate(AcceptEvents[::-1]):
        caseno,severity,ccoid,casetime = event.split('-~')
        if parser.parse(casetime).strftime("%Y-%m-%d") == date:
            RemovedDateAcceptEvents.pop(len(AcceptEvents)-index-1)
        if parser.parse(casetime) < parser.parse(date):
            break
    return RemovedDateAcceptEvents
date = (datetime.datetime.today() - datetime.timedelta(days=0)).strftime("%Y-%m-%d")
RemovedDateAcceptEvents = RemoveAcceptEventByDate(AcceptEvents,date)
RemovedDateAcceptEvents[-10:]

['689038611-~1-~kahande-~2020-05-07 03:38:59',
 '689038630-~3-~hethakur-~2020-05-07 03:48:03',
 '689032714-~3-~tonzeng-~2020-05-07 04:09:48',
 '689038712-~3-~raghb-~2020-05-07 04:16:36',
 '689038709-~2-~mkamijo-~2020-05-07 04:18:03',
 '689038718-~3-~minkwong-~2020-05-07 04:25:32',
 '688412150-~3-~jevidal-~2020-05-07 05:38:12',
 '688988005-~3-~reperuma-~2020-05-07 05:43:21',
 '687677961-~4-~knagavol-~2020-05-07 05:44:37',
 '689038922-~3-~tskanai-~2020-05-07 05:48:38']

In [259]:
AddEventList = [
    '688959554-~3-~siddhp-~2020-04-27 01:48:03',
    '688911513-~3-~zdazhi-~2020-04-27 05:06:32'
]

######Function to add a speicifc date inqueue inorder events ######
def AddAcceptEventByDate(AcceptEvents,AddList,date):
    ######Need to deep copy the list ######
    AddedDateAcceptEvents = [event for event in AcceptEvents]
    insert_index = len(AcceptEvents)
    for index,event in enumerate(AcceptEvents):
        caseno,severity,ccoid,casetime = event.split('-~')
        if parser.parse(casetime) > parser.parse(date):
            insert_index = index
            break
    print("Insert Index {}".format(insert_index))
    for event in AddList[::-1]:
        print("Inderting {} at {}".format(event,insert_index))
        AddedDateAcceptEvents.insert(insert_index,event)
    return AddedDateAcceptEvents

date = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
AddedDateAcceptEvents = AddAcceptEventByDate(AcceptEvents,AddEventList,date)
AddedDateAcceptEvents[-25:]

Insert Index 1915
Inderting 688911513-~3-~zdazhi-~2020-04-27 05:06:32 at 1915
Inderting 688959554-~3-~siddhp-~2020-04-27 01:48:03 at 1915


['689047226-~3-~pli3-~2020-05-08 03:16:09',
 '688949624-~1-~raghb-~2020-05-08 03:23:42',
 '688572925-~2-~jaehkim-~2020-05-08 03:35:44',
 '689047162-~3-~minkwong-~2020-05-08 03:36:54',
 '689047345-~4-~jawalia-~2020-05-08 03:53:12',
 '689042734-~3-~tonzeng-~2020-05-08 04:08:53',
 '688856354-~3-~gauvasud-~2020-05-08 04:13:33',
 '688771603-~3-~gauvasud-~2020-05-08 04:14:56',
 '688889442-~3-~deepaky-~2020-05-08 04:26:32',
 '689047453-~3-~hethakur-~2020-05-08 04:28:46',
 '689047486-~3-~maveer-~2020-05-08 04:37:49',
 '689047359-~3-~lchuan-~2020-05-08 04:40:50',
 '688608334-~3-~kahande-~2020-05-08 04:49:28',
 '688921797-~3-~kahande-~2020-05-08 04:48:48',
 '689017662-~3-~minkwong-~2020-05-08 05:11:59',
 '688938176-~3-~kahande-~2020-05-08 05:24:50',
 '688822797-~3-~knagavol-~2020-05-08 05:27:03',
 '689047630-~2-~raghb-~2020-05-08 05:30:25',
 '688938176-~3-~reperuma-~2020-05-08 05:32:12',
 '689047664-~3-~debabbar-~2020-05-08 05:38:12',
 '689047657-~3-~jiangygu-~2020-05-08 05:44:33',
 '688995998-~